In [1]:
from IPython.display import HTML,display
import bibtexparser
from pybtex.database.input import bibtex

filebib = 'pubs.bib'
parser = bibtex.Parser()
bib_data = parser.parse_file(filebib)

with open(filebib) as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

def make_cita(bibentry):
    text_latex = bibentry.rich_fields.get('author')
    persons_print =[person.split(",")[0] for person in str(text_latex).split("and ")]
    #print(persons_print,len(persons_print))
    if len(persons_print)==1:
        cita = persons_print[0]
    elif len(persons_print)==2:
        cita = " and ".join(persons_print) 
    else:
        cita = persons_print[0]+" et al."

    cita+=" "+ str(bibentry.rich_fields.get('year'))
    return cita

string = ""
for bibentry_key in sorted(bib_data.entries.keys(),key=lambda x: bib_data.entries[x].fields["year"]):
    bibentry = bib_data.entries[bibentry_key]
    # bibentry.rich_fields.get('title')
    #print(make_cita(bibentry),"--->",nombres[bibentry_key])
    if "url" in bibentry.fields:
        string+="<a href='%s'>[%s]</a></br>"%(bibentry.fields["url"],make_cita(bibentry))
    else:
        string+="[%s]</br>"%(make_cita(bibentry))
            

HTML(string)

In [4]:
from datetime import datetime

extra = dict()
for k,v in bib_database.entries_dict.items():
    bibentry = bib_data.entries[k]
    nueva_ent = dict(v)
    extra[k] = dict()
    extra[k]["abstract"] = v["abstract"]
    extra[k]["title"] = bibentry.rich_fields.get('title')
    
    copyifkeys = ["journal","url","doi","booktitle"]
    for ci in copyifkeys:  
        if ci in bibentry.fields:
            extra[k][ci] = bibentry.rich_fields.get(ci)
    
    
    nueva_ent.pop("abstract")
    nueva_bd = bibtexparser.bibdatabase.BibDatabase()
    if "file" in nueva_ent:
        nueva_ent.pop("file")
    nueva_bd.entries = [nueva_ent]
    extra[k]["bibtex"] = "{%% raw %%} %s {%% endraw %%}"%bibtexparser.dumps(nueva_bd).rstrip()
    authors_feos = bibentry.rich_fields.get('author')
    #print(authors_feos)
    authors = [ ]
    for aut in authors_feos.split("and"):
        surname, name = aut.split(",")
        nombre_bien = str(name).replace(" ","")+" "+str(surname).replace(" ","")
        if (nombre_bien == "Gonzalo Mateo-García") or (nombre_bien == "G. Mateo-García") or (nombre_bien == "G. Mateo-Garcia"):
            nombre_bien  = "<u>%s</u>"%nombre_bien
        authors.append(nombre_bien)
    extra[k]["authors"] = ", ".join(authors[:-1])+" and "+authors[-1]
    year_month = bibentry.fields["year"]+" "+bibentry.fields["month"]
    dat = datetime.strptime(year_month+" 01","%Y %B %d")
    extra[k]["date"] = dat
    
extra
    

{'gomez-chova_cloud_2017': {'abstract': 'Automatic cloud masking of Earth observation images is one of the first required steps in optical remote sensing data processing since the operational use and product generation from satellite image time series might be hampered by undetected clouds. The high temporal revisit of current and forthcoming missions and the scarcity of labeled data force us to cast cloud screening as an unsupervised change detection problem in the temporal domain. We introduce a cloud screening method based on detecting abrupt changes along the time dimension. The main assumption is that image time series follow smooth variations over land (background) and abrupt changes will be mainly due to the presence of clouds. The method estimates the background surface changes using the information in the time series. In particular, we propose linear and nonlinear least squares regression algorithms that minimize both the prediction and the estimation error simultaneously. The

In [5]:
import jinja2
import os

ordered_keys = sorted(extra.keys(),key=lambda k: extra[k]["date"],reverse=True)
extra["mateo-garcia_convolutional_2017"]["slides"] = "https://www.dropbox.com/s/v5mf77lcbzeqq35/1640_cnn_MateoGarcia.pdf?raw=1"
extra["mateo-garcia_cloud_2017"]["poster"] = "https://www.dropbox.com/s/mzx83weuvqpsalv/conference_poster_6.pdf?raw=1"
extra["mateo-garcia_advances_2017"]["poster"] = "https://www.dropbox.com/s/or8l3r15i8unvgx/conference_poster_6.pdf?raw=1"

def get_template(tpl_path):
    path, filename = os.path.split(tpl_path)
    return jinja2.Environment(
        loader=jinja2.FileSystemLoader(path or './')
    ).get_template(filename)

tpl = get_template("template_pubs.tpl")

with open("index_2.html", "w") as s:
    tpl.stream(extra=extra,ordered_keys=ordered_keys,absolute_rel_path="").dump(s)

display(HTML("<a href='http://localhost:4000/publications/index_2.html' target='blank'>link</a>"))
